In [111]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
URL = "/workspace/TreeConvolution/DS_WIKIDATA_TESIS/"

import torch.nn as nn
from model_trees_algebra import NeoRegression

class BaoTrainingException(Exception):
    pass


def split_ds(all_data, val_rate, seed):
    """test_rate is a rate of the total, val_rate is a rate of the (total - test_rate)"""
    ranges = {}
    ranges['1_2'] = all_data[(all_data["time"] > 0)    & (all_data["time"] <= 2)]
    ranges['2_3'] = all_data[(all_data["time"] > 2)    & (all_data["time"] <= 3)]
    ranges['3_4'] = all_data[(all_data["time"] > 3)    & (all_data["time"] <= 4)]
    ranges['4_5'] = all_data[(all_data["time"] > 4)    & (all_data["time"] <= 5)]
    ranges['5_8'] = all_data[(all_data["time"] > 5)    & (all_data["time"] <= 8)]
    ranges['8_10'] = all_data[(all_data["time"] > 8)   & (all_data["time"] <= 10)]
    ranges['10_20'] =   all_data[(all_data["time"] > 10) & (all_data["time"] <= 20)]
    ranges['20_30'] =   all_data[(all_data["time"] > 20) & (all_data["time"] <= 30)]
    ranges['30_40'] =   all_data[(all_data["time"] > 30) & (all_data["time"] <= 40)]
    ranges['40_50'] =   all_data[(all_data["time"] > 40) & (all_data["time"] <= 50)]
    ranges['50_60'] =   all_data[(all_data["time"] > 50) & (all_data["time"] <= 60)]
    ranges['60_80'] =   all_data[(all_data["time"] > 60) & (all_data["time"] <= 80)]
    ranges['80_100'] =  all_data[(all_data["time"] > 80) & (all_data["time"] <= 100)]
    ranges['100_150'] = all_data[(all_data["time"] > 100) & (all_data["time"] <= 150)]
    ranges['150_200'] = all_data[(all_data["time"] > 150) & (all_data["time"] <= 200)]
    ranges['200_250'] = all_data[(all_data["time"] > 200) & (all_data["time"] <= 250)]
    ranges['250_last'] = all_data[(all_data["time"] > 250)]
    train_data = []
    val_data = []
    test_data = []
    for rang in ranges.values():
        if rang.shape[0] >= 3:
            X_train, X_val = train_test_split(
                rang, test_size=val_rate, shuffle=True,random_state=seed)

            train_data.append(X_train)
            val_data.append(X_val)
    train_data_list = pd.concat(train_data)
    val_data_list = pd.concat(val_data)
    print("Shapes : Train: {} Val: {}".format(train_data_list.shape, val_data_list.shape))
    return train_data_list, val_data_list

# ds_test = pd.read_csv(URL + "ds_test_pred_filtered.csv", delimiter="ᶶ", engine='python')
data_train_val = pd.read_csv(URL + "ds_trainval_pred_filtered.csv", delimiter="ᶶ", engine='python')

data_train_val = data_train_val[data_train_val['time'] <=65]
# ds_test = ds_test[ds_test['time'] <=65]

print("Shape: train_val data" , data_train_val.shape)
import json
def clear_error_tuples(x):
    try:
        x_tree = json.loads(x)
        return True
    except:
        print("Error in data ignored!", x)
        return False
# ds_train, ds_val = split_ds(data_train_val, 0.5, seed=fold)
#Datasets
datasets_splits = []
for fold in range(5):
    # We use a fold bar as seed for reproductibility
    ds_train, ds_val = split_ds(data_train_val, 0.5, seed=fold)
    ds_train  = ds_train[ds_train['trees'].apply(lambda x: clear_error_tuples(x))]
    ds_val  = ds_val[ds_val['trees'].apply(lambda x: clear_error_tuples(x))]

    datasets_splits.append({"train": ds_train, "val" :ds_val})


Shape: train_val data (21703, 84)
Shapes : Train: (10850, 84) Val: (10853, 84)
Error in data ignored! [bgp "VAR_URI_VARᶲhttp://www.wikidata.org/prop/direct/P685"]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [datasetnames ?var1]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Shapes : Train: (10850, 84) Val: (10853, 84)
Error in data ignored! [datasetnames ?var1]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [bgp "VAR_URI_VARᶲhttp://www.wikidata.org/prop/direct/P685"]
Shapes : Train: (10850, 84) Val: (10853, 84)
Error in data ignored! [datasetnames ?var1]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [red

In [20]:
for fold, data in enumerate(datasets_splits):
    data['train'].to_pickle("./fold_data_"+str(fold)+"_train.pickle")
    data['val'].to_pickle("./fold_data_"+str(fold)+"_val.pickle")
# file_to_store = open("./folds_execution_baseline.pickle", "wb")
# pickle.dump(folds_execution, file_to_store)
# file_to_store.close()

In [112]:
from torch.utils.data import DataLoader
import json

def getpredictions_info(x_val_tree, x_val_query, y_val):
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=64, num_workers=0, shuffle=False, collate_fn=reg.collate_with_card)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}

def getmax(x):
    lista=  list(x.values())
    maximo = 0
    for el in lista:
        if (maximo < float(el)):
            maximo = float(el)
    return maximo

def getpreds(values, preds_dic):
    index = len(preds_dic)
    for x in values:
        lista=  list(json.loads(x).keys())
        for el in lista:
            if not el.startswith("http"):
                continue
            if el not in preds_dic:
                preds_dic[el] = index
                index +=1
    return preds_dic

def pred2index_dict(x, pred_to_index, maxcardinality):
    resp = {}
    x = json.loads(x)
    for el in x.keys():
        if el in pred_to_index:
            resp[pred_to_index[el]] = float(x[el])/maxcardinality
    return resp

def prepare_query_level_data(x_train_query, x_val_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)

    x_train_query =pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)

    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    return x_train_query, x_val_query


In [8]:
# def run_baseline(x_train_query, y_train, x_val_query, y_val):
#     #Scale x_query data.
#     xqtrain = x_train_query.drop(columns=['json_cardinality'])
#     xqval   = x_val_query.drop(columns=['json_cardinality'])

#     scalerx = StandardScaler()
#     x_train_scaled = scalerx.fit_transform(xqtrain)
#     x_val_scaled = scalerx.transform(xqval)

#     x_train_query =pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
#     x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)

#     preds_dic = {"other": 0}
#     preds_dic = getpreds(x_train_query['json_cardinality'], preds_dic)
#     preds_dic = getpreds(x_val_query['json_cardinality'], preds_dic)

#     maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()

#     def pred2index_dict(x, pred_to_index):
#         resp = {}
#         x = json.loads(x)
#         for el in x.keys():
#             if el in pred_to_index:
#                 resp[pred_to_index[el]] = float(x[el])/maxcardinality
#         return resp

#     x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, preds_dic))
#     x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, preds_dic))
#     y_train = ds_train['time'].values
#     y_val = ds_val['time'].values
#     return run_baseline(x_train_query.values, y_train, x_val_query.values, y_val, maxcardinality,  preds_dic)

### Hipotesis Test 5x2cv

### P1(TreeConv Neural Net)

In [ ]:
list_columns = ['assign', 'bgp', 'distinct', 'extend', 'filter', 'graph', 'group',
'join', 'leftjoin','minus', 'multi', 'notoneof', 'order', 'path*',  'project', 'reduced',
'sequence', 'slice', 'tolist', 'top','treesize', 'triple', 'union' ,
'path+', 'path?', 'pathN*', 'pathN+', 'pcs0', 'pcs1', 'pcs2', 'pcs3', 'pcs4',
'pcs5', 'pcs6', 'pcs7', 'pcs8', 'pcs9', 'pcs10', 'pcs11','pcs12', 'pcs13',
'pcs14', 'pcs15', 'pcs16', 'pcs17', 'pcs18', 'pcs19', 'pcs20', 'pcs21', 'pcs22', 'pcs23', 'pcs24',
                'filter_bound', 'filter_contains', 'filter_eq', 'filter_exists',
       'filter_ge', 'filter_gt', 'filter_isBlank', 'filter_isIRI',
       'filter_isLiteral', 'filter_lang', 'filter_langMatches', 'filter_le',
       'filter_lt', 'filter_ne', 'filter_not', 'filter_notexists', 'filter_or',
       'filter_regex', 'filter_sameTerm', 'filter_str', 'filter_strends',
       'filter_strstarts', 'filter_subtract', 'has_slice', 'max_slice_limit', 'max_slice_start','json_cardinality']
import pickle
folds_execution = {}

for fold, data in enumerate(datasets_splits):
    
    ds_train = data['train']
    ds_val = data['val']
    print("Size Train: {}, Val {}, Fold".format(ds_train.shape[0], ds_val.shape[0]), fold)
    
    x_train_query = ds_train[list_columns]
    x_val_query   = ds_val[list_columns]

    x_train_tree = ds_train['trees'].values
    x_val_tree = ds_val['trees'].values

    y_train = ds_train['time'].values
    y_val = ds_val['time'].values



    x_train_tree = ds_train['trees'].values
    x_val_tree = ds_val['trees'].values

    y_train = ds_train['time'].values
    y_val = ds_val['time'].values

    aec_dir = '/workspace/bao_server/aec_wikidata_neo.pth'
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()

    verbose=True
    reg = NeoRegression(
         aec={'train_aec': False, 'use_aec': True,'aec_file': aec_dir+str(fold)+'.pth', 'aec_epochs': 200},
         epochs=300,
         maxcardinality=maxcardinality,
         in_channels_neo_net=512,
         tree_units=[512, 256, 128],
         tree_units_dense=[64, 32],
         early_stop_patience=10,
         early_stop_initial_patience=180,
         tree_activation_tree=nn.LeakyReLU,
         tree_activation_dense=nn.ReLU,
        optimizer={'optimizer': "Adam",'args':{"lr":0.00015}},
        figimage_size=(18,18),
        start_history_from_epoch=3
    )

    #Fit the transformer tree data
    reg.fit_transform_tree_data(ds_train, ds_val, ds_val)
    print("Trees tranformed!!!")

    x_train_query, x_val_query =  prepare_query_level_data(x_train_query, x_val_query)

    #Fit model
    reg.fit(x_train_tree, x_train_query.values, y_train, x_val_tree, x_val_query.values, y_val)
    folds_execution[fold] = {"tree_model": getpredictions_info(x_val_tree, x_val_query, y_val)}

file_to_store = open("./folds_execution.pickle", "wb")
pickle.dump(folds_execution, file_to_store)
file_to_store.close()
#     run_baseline(x_train_query, y_train, x_val_query, y_val)
#     folds_execution[fold] = {"tree_model": getpreds(x_val_tree, x_val_query, y_val)}

Size Train: 10850, Val 10853, Fold 0
Model optimizer: Adam lr: 0.00015
Error in data ignored! [bgp "VAR_URI_VARᶲhttp://www.wikidata.org/prop/direct/P685"]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [datasetnames ?var1]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [datasetnames ?var1]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Error in data ignored! [reduced   ["VAR_VAR_VARᶲNONE"]]
Trees tranformed!!!
Error in data ignored! ('[bgp "VAR_URI_VARᶲhttp://www.wikidata.org/prop/direct/P685"]', array([0.0, -0.264113788729782, -0.9941186771667332, -0.3707414548887624,
       -0.7590541261977151, 0.0, -0.44046431789137874,
       -0.22141582907959934, -0.3894622358234045, 0.0, 0.0,
       -0.29011181998932284, -0.1726342423301504, -0.185446336439559

[2021-01-13 04:43:52,035] INFO:model_trees_algebra:==> Epoch 0,	TRAIN_LOSS: 2086.5621302382683	_TRAIN_RMSE: 45.67890246315325,	VAL_LOSS: 6036.669801370691,	VAL_RMSE: 77.69600891532777


Initial patience 180
2021-01-13 04:44:05.945951 Epoch 1, Training loss 0.00021315404043168758


[2021-01-13 04:44:15,813] INFO:model_trees_algebra:==> Epoch 1,	TRAIN_LOSS: 16068.253188944811	_TRAIN_RMSE: 126.76061371319093,	VAL_LOSS: 359.4679446878325,	VAL_RMSE: 18.95963988813692


Initial patience 179
2021-01-13 04:44:35.047039 Epoch 2, Training loss 0.00014207684503429952


...

Initial patience 168
2021-01-13 11:10:12.340964 Epoch 13, Training loss 5.543947311691863e-05


[2021-01-13 11:10:22,574] INFO:model_trees_algebra:==> Epoch 13,	TRAIN_LOSS: 28.01937738937503	_TRAIN_RMSE: 5.293333296645416,	VAL_LOSS: 42.67178583705789,	VAL_RMSE: 6.532364490523924
[2021-01-13 11:10:22,574] INFO:model_trees_algebra:==> Epoch 13,	TRAIN_LOSS: 28.01937738937503	_TRAIN_RMSE: 5.293333296645416,	VAL_LOSS: 42.67178583705789,	VAL_RMSE: 6.532364490523924
[2021-01-13 11:10:22,574] INFO:model_trees_algebra:==> Epoch 13,	TRAIN_LOSS: 28.01937738937503	_TRAIN_RMSE: 5.293333296645416,	VAL_LOSS: 42.67178583705789,	VAL_RMSE: 6.532364490523924
[2021-01-13 11:10:22,574] INFO:model_trees_algebra:==> Epoch 13,	TRAIN_LOSS: 28.01937738937503	_TRAIN_RMSE: 5.293333296645416,	VAL_LOSS: 42.67178583705789,	VAL_RMSE: 6.532364490523924
[2021-01-13 11:10:22,574] INFO:model_trees_algebra:==> Epoch 13,	TRAIN_LOSS: 28.01937738937503	_TRAIN_RMSE: 5.293333296645416,	VAL_LOSS: 42.67178583705789,	VAL_RMSE: 6.532364490523924


Initial patience 167
2021-01-13 11:10:41.901523 Epoch 14, Training loss 5.366842249153781e-05


[2021-01-13 11:10:52,045] INFO:model_trees_algebra:==> Epoch 14,	TRAIN_LOSS: 27.465081269769026	_TRAIN_RMSE: 5.240713813000003,	VAL_LOSS: 45.76701834096832,	VAL_RMSE: 6.765132544227668
[2021-01-13 11:10:52,045] INFO:model_trees_algebra:==> Epoch 14,	TRAIN_LOSS: 27.465081269769026	_TRAIN_RMSE: 5.240713813000003,	VAL_LOSS: 45.76701834096832,	VAL_RMSE: 6.765132544227668
[2021-01-13 11:10:52,045] INFO:model_trees_algebra:==> Epoch 14,	TRAIN_LOSS: 27.465081269769026	_TRAIN_RMSE: 5.240713813000003,	VAL_LOSS: 45.76701834096832,	VAL_RMSE: 6.765132544227668
[2021-01-13 11:10:52,045] INFO:model_trees_algebra:==> Epoch 14,	TRAIN_LOSS: 27.465081269769026	_TRAIN_RMSE: 5.240713813000003,	VAL_LOSS: 45.76701834096832,	VAL_RMSE: 6.765132544227668
[2021-01-13 11:10:52,045] INFO:model_trees_algebra:==> Epoch 14,	TRAIN_LOSS: 27.465081269769026	_TRAIN_RMSE: 5.240713813000003,	VAL_LOSS: 45.76701834096832,	VAL_RMSE: 6.765132544227668


Initial patience 166


### P1(Baseline Dense Model)

In [8]:
from torch.utils.data import DataLoader
def run_baseline(x_train, y_train, x_val, y_val, maxcardinality, preds2index):
    
    verbose=True
    reg = DenseRegression(
        verbose=True,
         epochs=300,
         query_input_size= len(x_train[0]),
         query_hidden_inputs=[260, 380, 240],
         query_output=1,
#          loss={'func':'CustomMSELoss', 'threshold': 20, 'weight_factor': 1},
         early_stop_patience=10,
         early_stop_initial_patience=80,
         activation_dense=nn.ReLU,
        optimizer={'optimizer': "Adam",'args':{"lr":0.00015}},
        figimage_size=(18,18),
        preds2index=preds2index
    )
    reg.fit(x_train, y_train, x_val, y_val)
    
    pairs_val = list(zip(x_val, y_val))
    dataset_val = DataLoader(pairs_val, batch_size=64, num_workers=0, shuffle=False, collate_fn=reg.collate)
    preds = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*preds)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    print("RMSE for baseline: in fold:{}, set VAL: {}".format(fold, rmseval))
#     reg.scatter_image(preds, y_val, "Scatter real latency vs prediction on Val dataset.", "neo_with_aec_scatter_test", max_refference=70, figsize =(15,15))
    return preds , mseval, rmseval, maeval , reg.history

In [ ]:

import pickle
from model_dense import DenseRegression

folds_executiondense = {}
for fold, data in enumerate(datasets_splits):
    
    ds_train = data['train']
    ds_val = data['val']
    
    list_columns = ['assign', 'bgp', 'distinct', 'extend', 'filter', 'graph', 'group',
    'join', 'leftjoin','minus', 'multi', 'notoneof', 'order', 'path*',  'project', 'reduced',
    'sequence', 'slice', 'tolist', 'top','treesize', 'triple', 'union' ,
    'path+', 'path?', 'pathN*', 'pathN+', 'pcs0', 'pcs1', 'pcs2', 'pcs3', 'pcs4',
    'pcs5', 'pcs6', 'pcs7', 'pcs8', 'pcs9', 'pcs10', 'pcs11','pcs12', 'pcs13',
    'pcs14', 'pcs15', 'pcs16', 'pcs17', 'pcs18', 'pcs19', 'pcs20', 'pcs21', 'pcs22', 'pcs23', 'pcs24',
                    'filter_bound', 'filter_contains', 'filter_eq', 'filter_exists',
           'filter_ge', 'filter_gt', 'filter_isBlank', 'filter_isIRI',
           'filter_isLiteral', 'filter_lang', 'filter_langMatches', 'filter_le',
           'filter_lt', 'filter_ne', 'filter_not', 'filter_notexists', 'filter_or',
           'filter_regex', 'filter_sameTerm', 'filter_str', 'filter_strends',
           'filter_strstarts', 'filter_subtract', 'has_slice', 'max_slice_limit', 'max_slice_start','json_cardinality']

    x_train_query = ds_train[list_columns]
    x_val_query   = ds_val[list_columns]

    y_train = ds_train['time'].values
    y_val = ds_val['time'].values

    import json
    def getmax(x):
        lista=  list(x.values())
        maximo = 0
        for el in lista:
            if (maximo < float(el)):
                maximo = float(el)
        return maximo

    def getpreds(values, preds_dic):
        index = len(preds_dic)
        for x in values:
            lista=  list(json.loads(x).keys())
            for el in lista:
                if not el.startswith("http"):
                    continue
                if el not in preds_dic:
                    preds_dic[el] = index
                    index +=1
        return preds_dic

    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)

    x_train_query =pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)

    preds_dic = {}
    preds_dic = getpreds(x_train_query['json_cardinality'], preds_dic)
    preds_dic = getpreds(x_val_query['json_cardinality'], preds_dic)

    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()

    def pred2index_dict(x, pred_to_index):
        resp = {}
        x = json.loads(x)
        for el in x.keys():
            if el in pred_to_index:
                resp[pred_to_index[el]] = float(x[el])/maxcardinality
        return resp

    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, preds_dic))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, preds_dic))
    y_train = ds_train['time'].values
    y_val = ds_val['time'].values
    preds , mseval, rmseval, maeval , history = run_baseline(x_train_query.values, y_train, x_val_query.values, y_val, maxcardinality,  preds_dic)
    y_pred_val, y_real_val = zip(*preds)
    folds_executiondense[fold] = {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": history}
    
    file_to_store = open("./folds_execution_baseline.pickle", "wb")
    pickle.dump(folds_executiondense, file_to_store)
    file_to_store.close()
    print("Fold {} baseline Saved!!".format(fold))
#     run_baseline(x_train_query, y_train, x_val_query, y_val)
#     folds_execution[fold] = {"tree_model": getpreds(x_val_tree, x_val_query, y_val)}

Model optimizer: Adam lr: 0.00015
Initial input channels of query model: 79
[Linear(in_features=1770, out_features=260, bias=True), ReLU(), Dropout(p=0.25, inplace=False), Linear(in_features=260, out_features=380, bias=True), ReLU(), Dropout(p=0.25, inplace=False), Linear(in_features=380, out_features=240, bias=True), ReLU(), Dropout(p=0.25, inplace=False), Linear(in_features=240, out_features=1, bias=True)]
Epochs to run: 4
2021-01-13 17:58:45.096713 Epoch 0, Training loss 0.0030035390732673904


### Model with Autoencoder for Tree Nodes, Algebra + Graph Pattern + Filters + Cardinality

In [8]:
file_to_store = open("./select_query_features3.pickle", "wb")
pickle.dump(select_query_features, file_to_store)
file_to_store.close()

### P2

In [ ]:
list_columns = ['assign', 'bgp', 'distinct', 'extend', 'filter', 'graph', 'group',
'join', 'leftjoin','minus', 'multi', 'notoneof', 'order', 'path*',  'project', 'reduced',
'sequence', 'slice', 'tolist', 'top','treesize', 'triple', 'union' ,
'path+', 'path?', 'pathN*', 'pathN+', 'pcs0', 'pcs1', 'pcs2', 'pcs3', 'pcs4',
'pcs5', 'pcs6', 'pcs7', 'pcs8', 'pcs9', 'pcs10', 'pcs11','pcs12', 'pcs13',
'pcs14', 'pcs15', 'pcs16', 'pcs17', 'pcs18', 'pcs19', 'pcs20', 'pcs21', 'pcs22', 'pcs23', 'pcs24',
                'filter_bound', 'filter_contains', 'filter_eq', 'filter_exists',
       'filter_ge', 'filter_gt', 'filter_isBlank', 'filter_isIRI',
       'filter_isLiteral', 'filter_lang', 'filter_langMatches', 'filter_le',
       'filter_lt', 'filter_ne', 'filter_not', 'filter_notexists', 'filter_or',
       'filter_regex', 'filter_sameTerm', 'filter_str', 'filter_strends',
       'filter_strstarts', 'filter_subtract', 'has_slice', 'max_slice_limit', 'max_slice_start','json_cardinality']
import pickle
folds_execution = {}

for fold, data in enumerate(datasets_splits):
    
    # Switch train -> val and val -> train For P2 of Hypotesis Test
    ds_train = data['val']
    ds_val = data['train']
    
    print("Size Train: {}, Val {}, Fold".format(ds_train.shape[0], ds_val.shape[0]), fold)
    
    x_train_query = ds_train[list_columns]
    x_val_query   = ds_val[list_columns]

    x_train_tree = ds_train['trees'].values
    x_val_tree = ds_val['trees'].values

    y_train = ds_train['time'].values
    y_val = ds_val['time'].values



    x_train_tree = ds_train['trees'].values
    x_val_tree = ds_val['trees'].values

    y_train = ds_train['time'].values
    y_val = ds_val['time'].values

    aec_dir = '/workspace/bao_server/aec_wikidata_neo.pth'
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()

    verbose=True
    reg = NeoRegression(
         aec={'train_aec': False, 'use_aec': True,'aec_file': aec_dir+str(fold)+'.pth', 'aec_epochs': 200},
         epochs=300,
         maxcardinality=maxcardinality,
         in_channels_neo_net=512,
         tree_units=[512, 256, 128],
         tree_units_dense=[64, 32],
         early_stop_patience=10,
         early_stop_initial_patience=180,
         tree_activation_tree=nn.LeakyReLU,
         tree_activation_dense=nn.ReLU,
        optimizer={'optimizer': "Adam",'args':{"lr":0.00015}},
        figimage_size=(18,18),
        start_history_from_epoch=3
    )

    #Fit the transformer tree data
    reg.fit_transform_tree_data(ds_train, ds_val, ds_val)
    print("Trees tranformed!!!")

    x_train_query, x_val_query =  prepare_query_level_data(x_train_query, x_val_query)

    #Fit model
    reg.fit(x_train_tree, x_train_query.values, y_train, x_val_tree, x_val_query.values, y_val)
    folds_execution[fold] = {"tree_model": getpredictions_info(x_val_tree, x_val_query, y_val)}

    file_to_store = open("./folds_execution_p2.pickle", "wb")
    pickle.dump(folds_execution, file_to_store)
    file_to_store.close()
#     run_baseline(x_train_query, y_train, x_val_query, y_val)
#     folds_execution[fold] = {"tree_model": getpreds(x_val_tree, x_val_query, y_val)}

Size Train: 10849, Val 10848, Fold 0
Model optimizer: Adam lr: 0.00015
Trees tranformed!!!
X_train loaded
X_val loaded
Transforming Trees
Activation function in treeConv layers: <class 'torch.nn.modules.activation.LeakyReLU'>
Activation function in tree dense output layers: <class 'torch.nn.modules.activation.ReLU'>
[BinaryTreeConvWithQData(
  (weights): Conv1d(2086, 512, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(512, 256, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(256, 128, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), DynamicPooling(), Linear(in_features=128, out_features=64, bias=True), ReLU(), Linear(in_features=64, out_features=32, bias=True), ReLU(), Linear(in_features=32, out_features=1, bias

[2021-01-13 23:16:44,859] INFO:model_trees_algebra:==> Epoch 0,	TRAIN_LOSS: 3108.80747368557	_TRAIN_RMSE: 55.75668097802783,	VAL_LOSS: 672.8763355411992,	VAL_RMSE: 25.93985997535837


Initial patience 180
2021-01-13 23:16:57.498354 Epoch 1, Training loss 0.00017907107103886695


[2021-01-13 23:17:06,966] INFO:model_trees_algebra:==> Epoch 1,	TRAIN_LOSS: 1661.2365551386463	_TRAIN_RMSE: 40.758269776066875,	VAL_LOSS: 2884.4677826579536,	VAL_RMSE: 53.707241435936304


Initial patience 179
2021-01-13 23:17:25.047506 Epoch 2, Training loss 0.00011118449681602857


...

Initial patience 97
2021-01-13 23:55:13.924898 Epoch 84, Training loss 3.670131767315268e-05


[2021-01-13 23:55:23,407] INFO:model_trees_algebra:==> Epoch 84,	TRAIN_LOSS: 18.503203210695293	_TRAIN_RMSE: 4.30153498308398,	VAL_LOSS: 34.78182480762561,	VAL_RMSE: 5.8976117884806225


Initial patience 96
2021-01-13 23:55:41.559182 Epoch 85, Training loss 3.633829468142589e-05


[2021-01-13 23:55:50,982] INFO:model_trees_algebra:==> Epoch 85,	TRAIN_LOSS: 17.416429809839688	_TRAIN_RMSE: 4.173299630968245,	VAL_LOSS: 34.48460445084555,	VAL_RMSE: 5.872359359818296


Initial patience 95


### P2 Baseline

In [ ]:

import pickle
from model_dense import DenseRegression

folds_executiondense = {}
for fold, data in enumerate(datasets_splits):
    
    ds_train = data['train']
    ds_val = data['val']
    
    list_columns = ['assign', 'bgp', 'distinct', 'extend', 'filter', 'graph', 'group',
    'join', 'leftjoin','minus', 'multi', 'notoneof', 'order', 'path*',  'project', 'reduced',
    'sequence', 'slice', 'tolist', 'top','treesize', 'triple', 'union' ,
    'path+', 'path?', 'pathN*', 'pathN+', 'pcs0', 'pcs1', 'pcs2', 'pcs3', 'pcs4',
    'pcs5', 'pcs6', 'pcs7', 'pcs8', 'pcs9', 'pcs10', 'pcs11','pcs12', 'pcs13',
    'pcs14', 'pcs15', 'pcs16', 'pcs17', 'pcs18', 'pcs19', 'pcs20', 'pcs21', 'pcs22', 'pcs23', 'pcs24',
                    'filter_bound', 'filter_contains', 'filter_eq', 'filter_exists',
           'filter_ge', 'filter_gt', 'filter_isBlank', 'filter_isIRI',
           'filter_isLiteral', 'filter_lang', 'filter_langMatches', 'filter_le',
           'filter_lt', 'filter_ne', 'filter_not', 'filter_notexists', 'filter_or',
           'filter_regex', 'filter_sameTerm', 'filter_str', 'filter_strends',
           'filter_strstarts', 'filter_subtract', 'has_slice', 'max_slice_limit', 'max_slice_start','json_cardinality']

    x_train_query = ds_train[list_columns]
    x_val_query   = ds_val[list_columns]

    y_train = ds_train['time'].values
    y_val = ds_val['time'].values

    import json
    def getmax(x):
        lista=  list(x.values())
        maximo = 0
        for el in lista:
            if (maximo < float(el)):
                maximo = float(el)
        return maximo

    def getpreds(values, preds_dic):
        index = len(preds_dic)
        for x in values:
            lista=  list(json.loads(x).keys())
            for el in lista:
                if not el.startswith("http"):
                    continue
                if el not in preds_dic:
                    preds_dic[el] = index
                    index +=1
        return preds_dic

    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)

    x_train_query =pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)

    preds_dic = {}
    preds_dic = getpreds(x_train_query['json_cardinality'], preds_dic)
    preds_dic = getpreds(x_val_query['json_cardinality'], preds_dic)

    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()

    def pred2index_dict(x, pred_to_index):
        resp = {}
        x = json.loads(x)
        for el in x.keys():
            if el in pred_to_index:
                resp[pred_to_index[el]] = float(x[el])/maxcardinality
        return resp

    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, preds_dic))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, preds_dic))
    y_train = ds_train['time'].values
    y_val = ds_val['time'].values
    preds , mseval, rmseval, maeval , history = run_baseline(x_train_query.values, y_train, x_val_query.values, y_val, maxcardinality,  preds_dic)
    y_pred_val, y_real_val = zip(*preds)
    folds_executiondense[fold] = {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": history}
    
    file_to_store = open("./folds_execution_baseline_p2.pickle", "wb")
    pickle.dump(folds_executiondense, file_to_store)
    file_to_store.close()
    print("Fold {} baseline Saved!!".format(fold))
#     run_baseline(x_train_query, y_train, x_val_query, y_val)
#     folds_execution[fold] = {"tree_model": getpreds(x_val_tree, x_val_query, y_val)}

Model optimizer: Adam lr: 0.00015
Initial input channels of query model: 79
[Linear(in_features=1770, out_features=260, bias=True), ReLU(), Dropout(p=0.25, inplace=False), Linear(in_features=260, out_features=380, bias=True), ReLU(), Dropout(p=0.25, inplace=False), Linear(in_features=380, out_features=240, bias=True), ReLU(), Dropout(p=0.25, inplace=False), Linear(in_features=240, out_features=1, bias=True)]
Epochs to run: 4
2021-01-14 11:40:19.247256 Epoch 0, Training loss 0.002767978251189912


[2021-01-14 11:40:22,159] INFO:model_dense:==> Epoch 0, 	TRAIN_LOSS: 365.08648681640625	_TRAIN_RMSE: 19.107236862182617, 	VAL_LOSS: 169.1260223388672, 	VAL_RMSE: 13.004846572875977
[2021-01-14 11:40:22,159] INFO:model_dense:==> Epoch 0, 	TRAIN_LOSS: 365.08648681640625	_TRAIN_RMSE: 19.107236862182617, 	VAL_LOSS: 169.1260223388672, 	VAL_RMSE: 13.004846572875977
[2021-01-14 11:40:22,159] INFO:model_dense:==> Epoch 0, 	TRAIN_LOSS: 365.08648681640625	_TRAIN_RMSE: 19.107236862182617, 	VAL_LOSS: 169.1260223388672, 	VAL_RMSE: 13.004846572875977
[2021-01-14 11:40:22,159] INFO:model_dense:==> Epoch 0, 	TRAIN_LOSS: 365.08648681640625	_TRAIN_RMSE: 19.107236862182617, 	VAL_LOSS: 169.1260223388672, 	VAL_RMSE: 13.004846572875977
[2021-01-14 11:40:22,159] INFO:model_dense:==> Epoch 0, 	TRAIN_LOSS: 365.08648681640625	_TRAIN_RMSE: 19.107236862182617, 	VAL_LOSS: 169.1260223388672, 	VAL_RMSE: 13.004846572875977
[2021-01-14 11:40:22,159] INFO:model_dense:==> Epoch 0, 	TRAIN_LOSS: 365.08648681640625	_TRAIN_

Initial patience 180
2021-01-14 11:40:25.645164 Epoch 1, Training loss 0.0010191654104471464


[2021-01-14 11:40:28,627] INFO:model_dense:==> Epoch 1, 	TRAIN_LOSS: 1394.906005859375	_TRAIN_RMSE: 37.34844207763672, 	VAL_LOSS: 149.72291564941406, 	VAL_RMSE: 12.23613166809082
[2021-01-14 11:40:28,627] INFO:model_dense:==> Epoch 1, 	TRAIN_LOSS: 1394.906005859375	_TRAIN_RMSE: 37.34844207763672, 	VAL_LOSS: 149.72291564941406, 	VAL_RMSE: 12.23613166809082
[2021-01-14 11:40:28,627] INFO:model_dense:==> Epoch 1, 	TRAIN_LOSS: 1394.906005859375	_TRAIN_RMSE: 37.34844207763672, 	VAL_LOSS: 149.72291564941406, 	VAL_RMSE: 12.23613166809082
[2021-01-14 11:40:28,627] INFO:model_dense:==> Epoch 1, 	TRAIN_LOSS: 1394.906005859375	_TRAIN_RMSE: 37.34844207763672, 	VAL_LOSS: 149.72291564941406, 	VAL_RMSE: 12.23613166809082
[2021-01-14 11:40:28,627] INFO:model_dense:==> Epoch 1, 	TRAIN_LOSS: 1394.906005859375	_TRAIN_RMSE: 37.34844207763672, 	VAL_LOSS: 149.72291564941406, 	VAL_RMSE: 12.23613166809082
[2021-01-14 11:40:28,627] INFO:model_dense:==> Epoch 1, 	TRAIN_LOSS: 1394.906005859375	_TRAIN_RMSE: 37.34

Initial patience 179
2021-01-14 11:40:37.537642 Epoch 2, Training loss 0.0008393345383565577


...

Initial patience 76
2021-01-14 12:01:26.331473 Epoch 105, Training loss 0.0005297447771857467


[2021-01-14 12:01:29,379] INFO:model_dense:==> Epoch 105, 	TRAIN_LOSS: 116.22589111328125	_TRAIN_RMSE: 10.780811309814453, 	VAL_LOSS: 124.25704956054688, 	VAL_RMSE: 11.147064208984375
[2021-01-14 12:01:29,379] INFO:model_dense:==> Epoch 105, 	TRAIN_LOSS: 116.22589111328125	_TRAIN_RMSE: 10.780811309814453, 	VAL_LOSS: 124.25704956054688, 	VAL_RMSE: 11.147064208984375
[2021-01-14 12:01:29,379] INFO:model_dense:==> Epoch 105, 	TRAIN_LOSS: 116.22589111328125	_TRAIN_RMSE: 10.780811309814453, 	VAL_LOSS: 124.25704956054688, 	VAL_RMSE: 11.147064208984375
[2021-01-14 12:01:29,379] INFO:model_dense:==> Epoch 105, 	TRAIN_LOSS: 116.22589111328125	_TRAIN_RMSE: 10.780811309814453, 	VAL_LOSS: 124.25704956054688, 	VAL_RMSE: 11.147064208984375
[2021-01-14 12:01:29,379] INFO:model_dense:==> Epoch 105, 	TRAIN_LOSS: 116.22589111328125	_TRAIN_RMSE: 10.780811309814453, 	VAL_LOSS: 124.25704956054688, 	VAL_RMSE: 11.147064208984375
[2021-01-14 12:01:29,379] INFO:model_dense:==> Epoch 105, 	TRAIN_LOSS: 116.22589

Initial patience 75
2021-01-14 12:01:38.479457 Epoch 106, Training loss 0.0005190694113319922


[2021-01-14 12:01:41,528] INFO:model_dense:==> Epoch 106, 	TRAIN_LOSS: 116.4173355102539	_TRAIN_RMSE: 10.78968620300293, 	VAL_LOSS: 122.53622436523438, 	VAL_RMSE: 11.069607734680176
[2021-01-14 12:01:41,528] INFO:model_dense:==> Epoch 106, 	TRAIN_LOSS: 116.4173355102539	_TRAIN_RMSE: 10.78968620300293, 	VAL_LOSS: 122.53622436523438, 	VAL_RMSE: 11.069607734680176
[2021-01-14 12:01:41,528] INFO:model_dense:==> Epoch 106, 	TRAIN_LOSS: 116.4173355102539	_TRAIN_RMSE: 10.78968620300293, 	VAL_LOSS: 122.53622436523438, 	VAL_RMSE: 11.069607734680176
[2021-01-14 12:01:41,528] INFO:model_dense:==> Epoch 106, 	TRAIN_LOSS: 116.4173355102539	_TRAIN_RMSE: 10.78968620300293, 	VAL_LOSS: 122.53622436523438, 	VAL_RMSE: 11.069607734680176
[2021-01-14 12:01:41,528] INFO:model_dense:==> Epoch 106, 	TRAIN_LOSS: 116.4173355102539	_TRAIN_RMSE: 10.78968620300293, 	VAL_LOSS: 122.53622436523438, 	VAL_RMSE: 11.069607734680176
[2021-01-14 12:01:41,528] INFO:model_dense:==> Epoch 106, 	TRAIN_LOSS: 116.4173355102539	_

Initial patience 74


### P1 NuSVR

In [ ]:
from sklearn.svm import NuSVR
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from time import time
import numpy as np
import pickle
log_transformer = preprocessing.FunctionTransformer(
    np.log, np.exp,
    validate=True)
scale_transformer = preprocessing.StandardScaler()

pipeline = Pipeline([("log", log_transformer),
                                    ("scale", scale_transformer)])

def baseline_svr(C, nu, x_train_query, y_train, x_val_query, y_val):
    
    sv = NuSVR(C=C, nu=nu)

    y_train_log_std = pipeline.fit_transform(y_train)

    t0=time()
    print("before train: Init time: {}".format(round(t0,3)))
    
    sv.fit(x_train_query, y_train_log_std)
    t1=time()
    print("after train, finish time: {}".format(round(t1,3)))
    print("training time {}",format(round(t1-t0,3)))
    y_train_hat_svr = pipeline.inverse_transform(sv.predict(x_train_query).reshape(-1, 1))
    y_val_hat_svr = pipeline.inverse_transform(sv.predict(x_val_query).reshape(-1, 1))

    #MSE for valid
    msetrain = mean_squared_error(y_train, y_train_hat_svr)
    maetrain = mean_absolute_error(y_train, y_train_hat_svr)
    rmsetrain = np.sqrt(msetrain)
    
    mseval = mean_squared_error(y_val, y_val_hat_svr)
    maeval = mean_absolute_error(y_val, y_val_hat_svr)
    rmseval = np.sqrt(mseval)

    print("Train MSE: {}, MAE {}, RMSE {}".format(msetrain, maetrain, rmsetrain))
    print("Val MSE: {}, MAE {}, RMSE {}".format(mseval, maeval, rmseval))

    return {
        "model": sv,
        "pred": y_val_hat_svr, "real" : y_val, 
        "mse": mseval,
        "mae" : maeval, 
        "rmse": rmseval, "train_time": round(t1,3)}

folds_executionsvr = {}
for fold, data in enumerate(datasets_splits):
    
    ds_train = data['val']
    ds_val = data['train']
    
    list_columns = ['assign', 'bgp', 'distinct', 'extend', 'filter', 'graph', 'group',
    'join', 'leftjoin','minus', 'multi', 'notoneof', 'order', 'path*',  'project', 'reduced',
    'sequence', 'slice', 'tolist', 'top','treesize', 'triple', 'union' ,
    'path+', 'path?', 'pathN*', 'pathN+', 'pcs0', 'pcs1', 'pcs2', 'pcs3', 'pcs4',
    'pcs5', 'pcs6', 'pcs7', 'pcs8', 'pcs9', 'pcs10', 'pcs11','pcs12', 'pcs13',
    'pcs14', 'pcs15', 'pcs16', 'pcs17', 'pcs18', 'pcs19', 'pcs20', 'pcs21', 'pcs22', 'pcs23', 'pcs24']

    x_train_query = ds_train[list_columns]
    x_val_query   = ds_val[list_columns]

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(x_train_query)
    x_val_scaled = scalerx.transform(x_val_query)

    x_train_query =pd.DataFrame(x_train_scaled, index=x_train_query.index, columns=x_train_query.columns)
    x_val_query =  pd.DataFrame(x_val_scaled,   index=x_val_query.index, columns=x_val_query.columns)

    y_train = ds_train['time'].values
    y_val = ds_val['time'].values
    
    C = 260
    nu = 0.4
    svr_stats = baseline_svr(C, nu, x_train_query.values, y_train.reshape(-1, 1), x_val_query.values, y_val.reshape(-1, 1))
    folds_executionsvr[fold] = svr_stats
    
    file_to_store = open("./folds_execution_baseline_nusvr_p2.pickle", "wb")
    pickle.dump(folds_executionsvr, file_to_store)
    file_to_store.close()
    print("Fold {} baseline Saved!!".format(fold))



before train: Init time: 1610848803.301


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


after train, finish time: 1610849498.643
training time {} 695.342
Train MSE: 102.7638758423884, MAE 5.123516224398302, RMSE 10.137251888080339
Val MSE: 316.3899785310255, MAE 5.912450095448392, RMSE 17.787354455652633
Fold 0 baseline Saved!!
before train: Init time: 1610849512.837


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


after train, finish time: 1610850167.631
training time {} 654.794
Train MSE: 102.49612470048699, MAE 5.153686395366533, RMSE 10.12403697644803
Val MSE: 230.9074777371726, MAE 5.7985142966326055, RMSE 15.195640089748526
Fold 1 baseline Saved!!
before train: Init time: 1610850181.731


...

after train, finish time: 1610850896.726
training time {} 714.995
Train MSE: 103.3714030002176, MAE 5.118221736782828, RMSE 10.167172812548117
Val MSE: 253.26232828648892, MAE 5.861319504820801, RMSE 15.914217803162332
Fold 2 baseline Saved!!
before train: Init time: 1610850910.692


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


after train, finish time: 1610851597.912
training time {} 687.22
Train MSE: 102.05290456042883, MAE 5.1091341197295685, RMSE 10.102123764854044
Val MSE: 148.65214881279576, MAE 5.670017832470089, RMSE 12.192298750145346
Fold 3 baseline Saved!!
before train: Init time: 1610851612.054


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


after train, finish time: 1610852253.86
training time {} 641.805


In [ ]:
from sklearn.svm import NuSVR
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from time import time
import numpy as np
import pickle
log_transformer = preprocessing.FunctionTransformer(
    np.log, np.exp,
    validate=True)
scale_transformer = preprocessing.StandardScaler()

pipeline = Pipeline([("log", log_transformer),
                                    ("scale", scale_transformer)])

def baseline_svr(C, nu, x_train_query, y_train, x_val_query, y_val):
    
    sv = NuSVR(C=C, nu=nu)

    y_train_log_std = pipeline.fit_transform(y_train)

    t0=time()
    print("before train: Init time: {}".format(round(t0,3)))
    
    sv.fit(x_train_query, y_train_log_std)
    t1=time()
    print("after train, finish time: {}".format(round(t1,3)))
    print("training time {}",format(round(t1-t0,3)))
    y_train_hat_svr = pipeline.inverse_transform(sv.predict(x_train_query).reshape(-1, 1))
    y_val_hat_svr = pipeline.inverse_transform(sv.predict(x_val_query).reshape(-1, 1))

    #MSE for valid
    msetrain = mean_squared_error(y_train, y_train_hat_svr)
    maetrain = mean_absolute_error(y_train, y_train_hat_svr)
    rmsetrain = np.sqrt(msetrain)
    
    mseval = mean_squared_error(y_val, y_val_hat_svr)
    maeval = mean_absolute_error(y_val, y_val_hat_svr)
    rmseval = np.sqrt(mseval)

    print("Train MSE: {}, MAE {}, RMSE {}".format(msetrain, maetrain, rmsetrain))
    print("Val MSE: {}, MAE {}, RMSE {}".format(mseval, maeval, rmseval))

    return {
        "model": sv,
        "pred": y_val_hat_svr, "real" : y_val, 
        "mse": mseval,
        "mae" : maeval, 
        "rmse": rmseval, "train_time": round(t1,3)}

folds_executionsvr = {}
for fold, data in enumerate(datasets_splits):
    
    ds_train = data['val']
    ds_val = data['train']
    
    list_columns = ['assign', 'bgp', 'distinct', 'extend', 'filter', 'graph', 'group',
    'join', 'leftjoin','minus', 'multi', 'notoneof', 'order', 'path*',  'project', 'reduced',
    'sequence', 'slice', 'tolist', 'top','treesize', 'triple', 'union' ,
    'path+', 'path?', 'pathN*', 'pathN+', 'pcs0', 'pcs1', 'pcs2', 'pcs3', 'pcs4',
    'pcs5', 'pcs6', 'pcs7', 'pcs8', 'pcs9', 'pcs10', 'pcs11','pcs12', 'pcs13',
    'pcs14', 'pcs15', 'pcs16', 'pcs17', 'pcs18', 'pcs19', 'pcs20', 'pcs21', 'pcs22', 'pcs23', 'pcs24']

    x_train_query = ds_train[list_columns]
    x_val_query   = ds_val[list_columns]

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(x_train_query)
    x_val_scaled = scalerx.transform(x_val_query)

    x_train_query =pd.DataFrame(x_train_scaled, index=x_train_query.index, columns=x_train_query.columns)
    x_val_query =  pd.DataFrame(x_val_scaled,   index=x_val_query.index, columns=x_val_query.columns)

    y_train = ds_train['time'].values
    y_val = ds_val['time'].values
    
    C = 260
    nu = 0.25
    svr_stats = baseline_svr(C, nu, x_train_query.values, y_train.reshape(-1, 1), x_val_query.values, y_val.reshape(-1, 1))
    folds_executionsvr[fold] = svr_stats
    
    file_to_store = open("./folds_execution_baseline_nusvr_p2.pickle", "wb")
    pickle.dump(folds_executionsvr, file_to_store)
    file_to_store.close()
    print("Fold {} baseline Saved!!".format(fold))



In [ ]:
# result_baseline_model = []
# dftable = pd.DataFrame(columns=[])
# for i in range(1,20):
#     C = 260
#     nu = random.randrange(10,50,5)/100
#     result_baseline_model
#     sv,dftable = baseline_svr(C, nu, dftable)
#     result_baseline_model.append(sv)

#

In [21]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

import model_trees_algebra_aec
import os
import shutil
import numpy as np
import pandas as pd
URL = "/workspace/TreeConvolution/DS_WIKIDATA_TESIS/"

import matplotlib.pyplot as plt
import torch.nn as nn
import json
import random
from model_dense import DenseRegression
from model import AECTrainig
from model_trees_algebra_aec import NeoRegression  as NeoWithAEC
from model_trees_algebra import NeoRegression
from featurize import SPARQLTreeFeaturizer
from early_stopping import EarlyStopping
from net import Autoencoder, NeoNet
from sklearn.metrics import mean_squared_error, mean_absolute_error
class BaoTrainingException(Exception):
    pass

from sklearn.model_selection import train_test_split
def split_ds(all_data, val_rate, seed):
    """test_rate is a rate of the total, val_rate is a rate of the (total - test_rate)"""
    ranges = {}
    ranges['1_2'] = all_data[(all_data["time"] > 0)    & (all_data["time"] <= 2)]
    ranges['2_3'] = all_data[(all_data["time"] > 2)    & (all_data["time"] <= 3)]
    ranges['3_4'] = all_data[(all_data["time"] > 3)    & (all_data["time"] <= 4)]
    ranges['4_5'] = all_data[(all_data["time"] > 4)    & (all_data["time"] <= 5)]
    ranges['5_8'] = all_data[(all_data["time"] > 5)    & (all_data["time"] <= 8)]
    ranges['8_10'] = all_data[(all_data["time"] > 8)   & (all_data["time"] <= 10)]
    ranges['10_20'] =   all_data[(all_data["time"] > 10) & (all_data["time"] <= 20)]
    ranges['20_30'] =   all_data[(all_data["time"] > 20) & (all_data["time"] <= 30)]
    ranges['30_40'] =   all_data[(all_data["time"] > 30) & (all_data["time"] <= 40)]
    ranges['40_50'] =   all_data[(all_data["time"] > 40) & (all_data["time"] <= 50)]
    ranges['50_60'] =   all_data[(all_data["time"] > 50) & (all_data["time"] <= 60)]
    ranges['60_80'] =   all_data[(all_data["time"] > 60) & (all_data["time"] <= 80)]
    ranges['80_100'] =  all_data[(all_data["time"] > 80) & (all_data["time"] <= 100)]
    ranges['100_150'] = all_data[(all_data["time"] > 100) & (all_data["time"] <= 150)]
    ranges['150_200'] = all_data[(all_data["time"] > 150) & (all_data["time"] <= 200)]
    ranges['200_250'] = all_data[(all_data["time"] > 200) & (all_data["time"] <= 250)]
    ranges['250_last'] = all_data[(all_data["time"] > 250)]
    train_data = []
    val_data = []
    test_data = []
    for rang in ranges.values():
        if rang.shape[0] >= 3:
            X_train, X_val = train_test_split(
                rang, test_size=val_rate, shuffle=True,random_state=seed)

            train_data.append(X_train)
            val_data.append(X_val)
    train_data_list = pd.concat(train_data)
    val_data_list = pd.concat(val_data)
    print("Shapes : Train: {} Val: {}".format(train_data_list.shape, val_data_list.shape))
    return train_data_list, val_data_list

# ds_test = pd.read_csv(URL + "ds_test_pred_filtered.csv", delimiter="ᶶ", engine='python')
data_train_val = pd.read_csv(URL + "ds_trainval_pred_filtered.csv", delimiter="ᶶ", engine='python')

data_train_val = data_train_val[data_train_val['time'] <=65]
# ds_test = ds_test[ds_test['time'] <=65]

print("Shape: train_val data" , data_train_val.shape)
import json
def clear_error_tuples(x):
    try:
        x_tree = json.loads(x)
        return True
    except:
        print("Error in data ignored!", x)
        return False
ds_train, ds_val = split_ds(data_train_val, 0.2, seed=fold)


from sklearn.svm import NuSVR
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from time import time
import numpy as np
import pickle
log_transformer = preprocessing.FunctionTransformer(
    np.log, np.exp,
    validate=True)
scale_transformer = preprocessing.StandardScaler()

pipeline = Pipeline([("log", log_transformer),
                                    ("scale", scale_transformer)])

def baseline_svr(C, nu, x_train_query, y_train, x_val_query, y_val):
    
    sv = NuSVR(C=C, nu=nu)

    y_train_log_std = pipeline.fit_transform(y_train)

    t0=time()
    print("before train: Init time: {}".format(round(t0,3)))
    
    sv.fit(x_train_query, y_train_log_std)
    t1=time()
    print("after train, finish time: {}".format(round(t1,3)))
    print("training time {}",format(round(t1-t0,3)))
    y_train_hat_svr = pipeline.inverse_transform(sv.predict(x_train_query).reshape(-1, 1))
    y_val_hat_svr = pipeline.inverse_transform(sv.predict(x_val_query).reshape(-1, 1))

    #MSE for valid
    msetrain = mean_squared_error(y_train, y_train_hat_svr)
    maetrain = mean_absolute_error(y_train, y_train_hat_svr)
    rmsetrain = np.sqrt(msetrain)
    
    mseval = mean_squared_error(y_val, y_val_hat_svr)
    maeval = mean_absolute_error(y_val, y_val_hat_svr)
    rmseval = np.sqrt(mseval)

    print("Train MSE: {}, MAE {}, RMSE {}".format(msetrain, maetrain, rmsetrain))
    print("Val MSE: {}, MAE {}, RMSE {}".format(mseval, maeval, rmseval))

    return {
        "C": C,
        "nu": nu,
        "model": sv,
        "pred": y_val_hat_svr, "real" : y_val, 
        "mse": mseval,
        "mae" : maeval, 
        "rmse": rmseval, "train_time": round(t1,3)}

folds_executionsvr = {}
for i in range(10):
    C = random.randrange(200, 300, 20)
    nu = random.randrange(10, 50, 5)/100
    print(f"NuSVR with Hyperparams C: {C} nu: {nu}")
    print(f"Size Train {ds_train.shape[0]} Val: {ds_val.shape[0]}")
    
    list_columns = ['assign', 'bgp', 'distinct', 'extend', 'filter', 'graph', 'group',
    'join', 'leftjoin','minus', 'multi', 'notoneof', 'order', 'path*',  'project', 'reduced',
    'sequence', 'slice', 'tolist', 'top','treesize', 'triple', 'union' ,
    'path+', 'path?', 'pathN*', 'pathN+', 'pcs0', 'pcs1', 'pcs2', 'pcs3', 'pcs4',
    'pcs5', 'pcs6', 'pcs7', 'pcs8', 'pcs9', 'pcs10', 'pcs11','pcs12', 'pcs13',
    'pcs14', 'pcs15', 'pcs16', 'pcs17', 'pcs18', 'pcs19', 'pcs20', 'pcs21', 'pcs22', 'pcs23', 'pcs24']

    x_train_query = ds_train[list_columns]
    x_val_query   = ds_val[list_columns]

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(x_train_query)
    x_val_scaled = scalerx.transform(x_val_query)

    x_train_query =pd.DataFrame(x_train_scaled, index=x_train_query.index, columns=x_train_query.columns)
    x_val_query =  pd.DataFrame(x_val_scaled,   index=x_val_query.index, columns=x_val_query.columns)

    y_train = ds_train['time'].values
    y_val = ds_val['time'].values
    
#     C = 260
#     nu = 0.4
    svr_stats = baseline_svr(C, nu, x_train_query.values, y_train.reshape(-1, 1), x_val_query.values, y_val.reshape(-1, 1))
    folds_executionsvr[i] = svr_stats
    
    file_to_store = open("./baseline_nusvr_fithparams.pickle", "wb")
    pickle.dump(folds_executionsvr, file_to_store)
    file_to_store.close()
    print("Fold {} baseline Saved!!".format(fold))



Shape: train_val data (21703, 84)
Shapes : Train: (17357, 84) Val: (4346, 84)
NuSVR with Hyperparams C: 240 nu: 0.35
Size Train 17357 Val: 4346
before train: Init time: 1610718033.055


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


after train, finish time: 1610718958.495
training time {} 925.44
Train MSE: 105.29148118086134, MAE 5.359197090408729, RMSE 10.261163734238984
Val MSE: 115.9290453104456, MAE 5.581038027336509, RMSE 10.76703512163147
Fold 4 baseline Saved!!
NuSVR with Hyperparams C: 200 nu: 0.4
Size Train 17357 Val: 4346
before train: Init time: 1610718977.537


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


after train, finish time: 1610720125.224
training time {} 1147.687
Train MSE: 105.19270247271025, MAE 5.214462636191637, RMSE 10.25634937356905
Val MSE: 115.68745751587717, MAE 5.465129446844378, RMSE 10.755810407211404
Fold 4 baseline Saved!!
NuSVR with Hyperparams C: 220 nu: 0.35
Size Train 17357 Val: 4346
before train: Init time: 1610720146.761


...

after train, finish time: 1610724011.141
training time {} 961.778
Train MSE: 105.3090807576363, MAE 5.363283722023659, RMSE 10.26202128031492
Val MSE: 115.6034496588649, MAE 5.583360195485378, RMSE 10.751904466598692
Fold 4 baseline Saved!!
NuSVR with Hyperparams C: 260 nu: 0.3
Size Train 17357 Val: 4346
before train: Init time: 1610724030.405


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


after train, finish time: 1610724867.642
training time {} 837.237
Train MSE: 105.75280776442608, MAE 5.548674358667388, RMSE 10.283618417873452
Val MSE: 114.38325372933276, MAE 5.7396132806434705, RMSE 10.695010693278093
Fold 4 baseline Saved!!
NuSVR with Hyperparams C: 220 nu: 0.4
Size Train 17357 Val: 4346
before train: Init time: 1610724884.296


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


after train, finish time: 1610726107.717
training time {} 1223.421
Train MSE: 105.2273275715987, MAE 5.211324870159444, RMSE 10.258037218279075
Val MSE: 116.18333230398207, MAE 5.466077743075228, RMSE 10.77883724267057
Fold 4 baseline Saved!!


In [101]:
tune_hp_svr2 = pd.DataFrame(data=data).sort_values('rmseval')

In [102]:
tune_hp_svr2 = tune_hp_svr2[['C' , 'nu', 'train_time','mse', 'rmse', 'mae',  'mseval', 'rmseval', 'maeval']]

In [106]:
print(tune_hp_svr2.round(2).to_latex(index = False))

\begin{tabular}{lllllllll}
\toprule
   C &    nu & train\_time &                 mse &                rmse &                mae &              mseval &             rmseval &              maeval \\
\midrule
 260 &  0.25 &    649.364 &  107.05652405323015 &  10.346812265293604 &  5.744831951679673 &  112.25473332184852 &  10.595033427122754 &   5.913288336358367 \\
 260 &   0.3 &    837.237 &  105.75280776442608 &  10.283618417873452 &  5.548674358667388 &  114.38325372933276 &  10.695010693278093 &  5.7396132806434705 \\
 220 &  0.35 &    967.054 &   105.3090807576363 &   10.26202128031492 &  5.363283722023659 &   115.6034496588649 &  10.751904466598692 &   5.583360195485378 \\
 220 &  0.35 &    961.778 &   105.3090807576363 &   10.26202128031492 &  5.363283722023659 &   115.6034496588649 &  10.751904466598692 &   5.583360195485378 \\
 200 &   0.4 &   1147.687 &  105.19270247271025 &   10.25634937356905 &  5.214462636191637 &  115.68745751587717 &  10.755810407211404 &   5.4651294468443

In [108]:
tune_hp_svr2.to_pickle("./hparameters_tune_nusvr.pickle")

In [109]:
tune_hp_svr2

,C,nu,train_time,mse,rmse,mae,mseval,rmseval,maeval
6,260,0.25,649.364,107.05652405323015,10.346812265293604,5.744831951679673,112.25473332184852,10.595033427122754,5.913288336358367
8,260,0.3,837.237,105.75280776442608,10.283618417873452,5.548674358667388,114.38325372933276,10.695010693278093,5.7396132806434705
2,220,0.35,967.054,105.3090807576363,10.26202128031492,5.363283722023659,115.6034496588649,10.751904466598692,5.583360195485378
7,220,0.35,961.778,105.3090807576363,10.26202128031492,5.363283722023659,115.6034496588649,10.751904466598692,5.583360195485378
1,200,0.4,1147.687,105.19270247271025,10.25634937356905,5.214462636191637,115.68745751587717,10.755810407211404,5.465129446844378
0,240,0.35,925.44,105.29148118086134,10.261163734238984,5.359197090408729,115.9290453104456,10.76703512163147,5.581038027336509
5,280,0.2,510.775,111.2434208978989,10.54719967090312,5.999285361415668,116.0670341741115,10.773441148217755,6.153880948968004
9,220,0.4,1223.421,105.2273275715987,10.258037218279075,5.211324870159444,116.18333230398207,10.77883724267057,5.466077743075228
3,220,0.2,439.755,111.34332372172473,10.551934596164095,6.004116177519384,116.25924319044537,10.782357960596809,6.161372813163404
4,220,0.15,270.116,119.52551486598352,10.932772515056898,6.378254445687964,124.36404760136784,11.15186296550347,6.512090711364716


In [83]:
import re 
text.split("Fold 4 baseline Saved!!")[0].split(", MSE ")

['Shape: train_val data (21703, 84)\nShapes : Train: (17357, 84) Val: (4346, 84)\nNuSVR with Hyperparams C: 240 nu: 0.35\nSize Train 17357 Val: 4346\nbefore train: Init time: 1610718033.055\n/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().\n  return f(**kwargs)\nafter train, finish time: 1610718958.495\ntraining time {} 925.44\nTrain MSE: 105.29148118086134, MAE 5.359197090408729, RMSE 10.261163734238984\nVal MSE: 115.9290453104456, MAE 5.581038027336509, RMSE 10.76703512163147\n']

In [96]:
data

[{'C': '240',
  'nu': '0.35',
  'mse': '105.29148118086134',
  'mae': '5.359197090408729',
  'rmse': '10.261163734238984',
  'mseval': '115.9290453104456',
  'maeval': '5.581038027336509',
  'rmseval': '10.76703512163147'},
 {'C': '200',
  'nu': '0.4',
  'mse': '105.19270247271025',
  'mae': '5.214462636191637',
  'rmse': '10.25634937356905',
  'mseval': '115.68745751587717',
  'maeval': '5.465129446844378',
  'rmseval': '10.755810407211404'},
 {'C': '220',
  'nu': '0.35',
  'mse': '105.3090807576363',
  'mae': '5.363283722023659',
  'rmse': '10.26202128031492',
  'mseval': '115.6034496588649',
  'maeval': '5.583360195485378',
  'rmseval': '10.751904466598692'},
 {'C': '220',
  'nu': '0.2',
  'mse': '111.34332372172473',
  'mae': '6.004116177519384',
  'rmse': '10.551934596164095',
  'mseval': '116.25924319044537',
  'maeval': '6.161372813163404',
  'rmseval': '10.782357960596809'},
 {'C': '220',
  'nu': '0.15',
  'mse': '119.52551486598352',
  'mae': '6.378254445687964',
  'rmse': '10

In [100]:
data = []
for texto in text.split("Fold 4 baseline Saved!!"):
    params = re.search("NuSVR with Hyperparams C: (.+?) nu: (.+?)\n", texto)
    train = re.search("Train MSE: (.+?), MAE (.+?), RMSE (.+?)\n", texto)
    val = re.search("Val MSE: (.+?), MAE (.+?), RMSE (.+?)\n", texto)
    ttime = re.search("training time {} (.+?)\n", texto)
    try:
        data.append({"C": params.group(1), "nu": params.group(2), "train_time": ttime.group(1),
                     "mse": train.group(1),"mae": train.group(2),"rmse": train.group(3), 
                     "mseval": val.group(1),"maeval": val.group(2),"rmseval": val.group(3), 
                    })
    except:
        break